In [ ]:
Задание 1.
Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = [‘python’, ‘парсинг’]

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [12]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time

In [15]:
head_url = 'https://habr.com'                                                           # голова адреса страницы
url = head_url + '/ru/all/'                                                             # адрес страницы
keywords = ['python', 'парсинг']                                                        # список слов для поиска
headers = {'User-Agent': 
           'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.70'
          }
keywords_search = '|'.join(keywords).lower()                                            # строка поиска в формате RegExp
params = {}                                                                             # словарь параметров
df_result = pd.DataFrame()                                                              # DataFrame результатов поиска

def find_item(head_url, post):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.70'}
    header = post.find('a', class_='tm-article-snippet__title-link')                    # делим пост на заголовок
    post_header = header.text
    post_review = post.find('div', class_='article-formatted-body').text                #                        и обзор
    post_link   = head_url + header.attrs.get('href')                                   # из заголовка получаем ссылку  
    req = requests.get(post_link, headers=headers)                                      # по ссылке запрашиваем содержимое
    if req.ok:                                                                          # если содержимое получено
        soup =  BeautifulSoup(req.text, 'html.parser')                                  
        post_body = soup.find('div', class_='tm-article-body').text                     # то достаем из него текст статьи
    else:
        post_body =''
    if (re.search(fr"{keywords_search}",                                                # если находим ключевые слова
                  post_header                                                           # в заголовке
                  + post_review                                                         # обзоре
                  + post_body)):                                                        # или самом тексте статьи,
        post_date = (post.find('span', class_='tm-article-snippet__datetime-published') # то получаем дату из поста    
                     .time                                                              
                     .attrs                                                             
                     .get('datetime'))                                                                    
        return {'Дата': post_date,                                                      # и возвращаем результат в виде словаря
                'Заголовок': str(post_header).strip(), 
                'Ссылка': post_link, 
                'Текст статьи': str(post_body).strip()}
    else:                                                                               # если совпадений не найдено,
        return False                                                                    # то возвращаем False
    
req = requests.get(url, headers=headers)                                                # читаем содержимое с заданной страницы
soup = BeautifulSoup(req.text, 'html.parser') 
page_count = int(soup.find_all('a', class_="tm-pagination__page")[-1].text.strip())     # ищем число страниц
for page in range(1, page_count + 1):                                                   # проходим по каждой странице 
    params['page'] = page                                                               # в параметры передаем номер текущей страницы
    req = requests.get(url, params, headers=headers)                                    # запрашиваем содержимое текущей страницы
    if req.ok:                                                                          # если результат запроса удачный
        soup = BeautifulSoup(req.text, 'html.parser')                                   # читаем текст в суп
        posts = soup.find_all('div', class_='tm-article-snippet')                       # разбиваем на посты
        for post in posts:                                                              # проходим по каждому посту
            item = find_item(head_url, post)                                            # передаем шапку адреса и пост, получаем словарь или False
            time.sleep(0.3)
            if item:                                                                    # если вернулся словарь, то добавляем в DF
                df_result = df_result.append(item, ignore_index=True)

KeyboardInterrupt: 

In [16]:
df_result

Дата      Заголовок  \
0  2021-08-09T13:16:17.000Z  Postgresso 33   

                                              Ссылка  \
0  https://habr.com/ru/company/postgrespro/blog/5...   

                                        Текст статьи  
0  Продолжаем экспериментировать с форматом: в эт...

In [ ]:
Задание 2.
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. 
Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: - <дата утечки> - <источник утечки> - <описание утечки>.

In [17]:
import requests
import pandas as pd

In [18]:
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 YaBrowser/21.6.1.274 Yowser/2.5 Safari/537.36',
           'Vaar-Header-App-Build-Version': '1.0.0',
           'Vaar-Header-App-Product': 'hackcheck-web-avast',
           'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
           'Vaar-Version': '0'           
          
          }
EMAIL = ['xxx@x.ru', 'yyy@y.com']
        
jsn ={'emailAddresses': EMAIL}
res = requests.post(url, json=jsn, headers=headers)
if res.status_code == 200:                                # если успешно, обрабатываем json
    df_breaches = pd.DataFrame(res.json()['breaches']).T.loc[:, ['publishDate', 'site', 'description']]
    df_breaches.rename(columns={'publishDate': 'дата утечки', 'site': 'источник утечки', 'description': 'описание утечки'}, inplace=True)
else: 
    print(res)                                            # иначе выводим ошибку

In [19]:
df_breaches

дата утечки       источник утечки  \
2      2016-10-21T00:00:00Z          linkedin.com   
3      2016-10-21T00:00:00Z             adobe.com   
12     2016-10-29T00:00:00Z                vk.com   
15     2016-10-23T00:00:00Z             imesh.com   
41     2016-10-24T00:00:00Z           dropbox.com   
2961   2017-01-31T00:00:00Z      cdprojektred.com   
3164   2017-02-14T00:00:00Z         cfire.mail.ru   
3176   2017-02-14T00:00:00Z        parapa.mail.ru   
3520   2017-03-15T00:00:00Z        globalreach.eu   
3587   2017-03-01T00:00:00Z          rayli.com.cn   
3669   2017-03-24T00:00:00Z             youku.com   
13094  2017-11-04T00:00:00Z        myheritage.com   
13662  2018-02-18T00:00:00Z            netlog.com   
16768  2019-06-13T00:00:00Z             canva.com   
17009  2019-10-17T00:00:00Z             zynga.com   
17110  2020-01-03T00:00:00Z         azcentral.com   
17670  2020-05-28T00:00:00Z           wishbone.io   
37177  2021-02-11T00:00:00Z  forums.vkmonline.com   

                                         описание утечки  
2      In 2012, online professional networking platfo...  
3      In October of 2013, criminals penetrated Adobe...  
12     Popular Russian social networking platform VKo...  
15     In June 2016, a cache of over 51 million user ...  
41     Cloud storage company Dropbox suffered a major...  
2961   In March 2016, CDProjektRed.com.com's forum da...  
3164   In July and August of 2016, two criminals carr...  
3176   In July and August 2016, two criminals execute...  
3520   In 2016, Global Reach Technology's database wa...  
3587   On an unconfirmed date, Chinese gossip site Ra...  
3669   Youku is a large Chinese video content company...  
13094  In October 2017, a customer database belonging...  
13662  Netlog (formerly known as Facebox and Bingbox)...  
16768  In May 2019, graphic-design site Canva's datab...  
17009  In September 2019, the game developer Zynga wa...  
17110  At an unconfirmed date, online Arizona newspap...  
17670  In January 2020, the online poll website Wishb...  
37177  At an unconfirmed date, the Russian-language m...